In [15]:
# import relevant libraries
import os
import pandas as pd
from rdkit import Chem
import rdkit.Chem.Descriptors
import numpy as np
import math

In [16]:
# import all data frames from the data folder and name the dataframe like the file name
# for example, the dataframe for the file 'data/1.csv' is called df_1

for file in os.listdir('../single_data_SMILES'):
    if file.endswith('.csv'):
        globals()['df_' + file[:-4]] = pd.read_csv('../single_data_SMILES/' + file, sep=';', encoding='utf-8', header=0)
        print(file)

experimental_hazards.csv
enthalpy_of_combustion.csv
thermal_conductivity_gas.csv
helmholtz_free_energy_of_formation.csv
entropy_of_gas.csv
henry_constant.csv
enthalpy_of_vaporization.csv
refractive_index.csv
gibbs_free_energy_of_formation.csv
enthalpy_of_fusion.csv
thermal_expansion_liq.csv
adsorption_capacity_gas.csv
adsorption_capacity_liq.csv
viscosity_gas.csv
solubility_water.csv
internal_energy_of_formation.csv
critical_values.csv
solubility_octanol_water.csv
solubility_parameters.csv
heat_capacity_gas.csv
density.csv
thermal_conductivity_liq.csv
diffusion_coefficient_water_infinite_dilution.csv
heat_capacity_liq.csv
heat_capacity_sol.csv
viscosity_liq.csv
entropy_of_formation.csv
diffusion_coefficient_air.csv
antoine_coefficients_mixed.csv
surface_tension.csv
enthalpy_of_formation.csv


In [17]:
# import our name dictionary
name_dict = pd.read_csv('../names_to_SMILES/name_dict.csv', sep=';', encoding='utf-8', header=0)
# set first column as name and second column as SMILES
name_dict.columns = ['Name', 'SMILES']

# convert to dictionary
name_dict = name_dict.set_index('Name').T.to_dict('list')
# remove the brackets from the SMILES
for key in name_dict:
    name_dict[key] = name_dict[key][0]

In [18]:
# import vectors and save as numpy arrays
T_vector = np.loadtxt('../T_vector.txt').astype(float)

print(T_vector)

[ 148.    150.    160.    170.    180.    186.15  190.    199.95  200.
  210.    220.    230.    231.    231.15  233.    233.15  238.65  240.
  248.    250.    260.    270.    273.15  274.5   275.15  278.15  279.5
  280.    283.15  287.15  287.65  288.    288.05  288.15  289.    289.15
  289.65  290.    290.15  291.05  291.15  292.05  293.    293.05  293.15
  293.65  293.8   294.    294.15  294.25  295.    295.15  296.    296.15
  296.65  296.75  297.    297.15  297.65  297.8   297.95  298.    298.15
  298.25  299.15  299.75  299.85  300.    300.15  300.65  301.15  303.
  303.15  307.    307.15  307.25  308.    308.15  309.25  310.    310.15
  313.    315.    318.    318.15  320.    320.15  323.    323.15  324.35
  328.    330.    331.15  333.15  334.15  337.5   338.    338.15  340.
  343.15  347.15  348.    348.15  349.15  350.    351.    351.15  352.15
  353.    353.15  357.65  359.15  360.    363.15  370.    372.15  373.
  373.15  380.    384.    386.    386.3   388.    390.    400.

In [5]:
### This Is Already Done - No Need To Run This Cell Again
# create full data frame and call first column 'SMILES'
df_full = pd.DataFrame(columns=['SMILES'])
# fill it with the SMILES from the name dictionary
df_full['SMILES'] = name_dict.values()
df_full['MW'] = 0

# for each SMILES in the first column, calculate the molecular weight
for i in range(len(df_full)):
    try:
        df_full['MW'][i] = Chem.Descriptors.MolWt(Chem.MolFromSmiles(df_full['SMILES'][i]))
    except:
        # drop the row if the SMILES is not valid
        df_full = df_full.drop(i)

# save as df_full.csv
df_full.to_csv('df_MW.csv', sep=';', encoding='utf-8', index=False)

/tmp/ipykernel_3463604/4084107574.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['MW'][i] = Chem.Descriptors.MolWt(Chem.MolFromSmiles(df_full['SMILES'][i]))
[13:00:29] SMILES Parse Error: syntax error while parsing: None
[13:00:29] SMILES Parse Error: Failed parsing SMILES 'None' for input: 'None'
/tmp/ipykernel_3463604/4084107574.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['MW'][i] = Chem.Descriptors.MolWt(Chem.MolFromSmiles(df_full['SMILES'][i]))
[13:00:29] SMILES Parse Error: syntax error while parsing: None
[13:00:29] SMILES Parse Error: Failed parsing SMILES 'None' for input: 'None'


In [19]:
# load df_full.csv
df_SMILES_MW = pd.read_csv('df_MW.csv', sep=';', encoding='utf-8', header=0)

print(df_SMILES_MW)

                     SMILES      MW
0                CCOC(C)OCC  118.18
1                      CC=O   44.05
2                   CC(N)=O   59.07
3           CC(=O)Nc1ccccc1  135.17
4                   CC(=O)O   60.05
...                     ...     ...
10139            CCOS(=O)OC  124.16
10140      CCCOS(=O)(=O)OCC  168.21
10141  CCCCCCCCOS(=O)(=O)OC  224.32
10142     CCCCCOS(=O)(=O)OC  182.24
10143       CCCOS(=O)(=O)OC  154.19

[10144 rows x 2 columns]


In [20]:
# duplicate each row len(T_vector) times and add the T_vector as a new column
df_full = pd.concat([df_SMILES_MW]*len(T_vector), ignore_index=True)
# add T_vector as a new column and fill it with the T_vector for each SMILES
df_full['T'] = np.repeat(T_vector, len(df_full)/len(T_vector))

# order by SMILES and T
df_full = df_full.sort_values(by=['SMILES', 'T'])
# delete duplicates
df_full = df_full.drop_duplicates()
# reset index
df_full = df_full.reset_index(drop=True)

# set MW to -np.inf when T!=298.15
df_full['MW'][df_full['T'] != 298.15] = -np.inf

print(df_full)

# save as df_SMILES_MW_T_X_Y.csv
df_full.to_csv('../df_SMILES_MW_T.csv', sep=';', encoding='utf-8', index=False)

/tmp/ipykernel_1859018/467649956.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['MW'][df_full['T'] != 298.15] = -np.inf


                SMILES   MW       T
0        BrC(Br)(Br)Br -inf   148.0
1        BrC(Br)(Br)Br -inf   150.0
2        BrC(Br)(Br)Br -inf   160.0
3        BrC(Br)(Br)Br -inf   170.0
4        BrC(Br)(Br)Br -inf   180.0
...                ...  ...     ...
1151677        c1cocn1 -inf  1450.0
1151678        c1cocn1 -inf  1460.0
1151679        c1cocn1 -inf  1470.0
1151680        c1cocn1 -inf  1480.0
1151681        c1cocn1 -inf  1490.0

[1151682 rows x 3 columns]


In [21]:
# load df_SMILES_MW_T_X_Y.csv
df_full = pd.read_csv('../df_SMILES_MW_T.csv', sep=';', encoding='utf-8', header=0)
# in second round, should have 1151682 rows

## Add the relevant columns of physicochemical data for given SMILES, T, X, Y
go through the list alphabetically, comment important equation, units and apply to each row

1. adsorption capacity of gas
we want to get Y as a function of K, n_invers, Ymin_ppmv, Ymax_ppmv

Q, Adsorption Capacity of Activated Carbon - Gas
logQ_gas = logK + 1/n logX
Q g/100g C, Y ppmv

In [83]:
# def adsorption_capacity_gas(df_full, df_adsorption_capacity_gas):
#     # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
#     #df_adsorption_capacity_gas = df_adsorption_capacity_gas.drop_duplicates(subset='SMILES')
#     merged_df = pd.merge(df_full, df_adsorption_capacity_gas, on='SMILES', how='left')

#     # drop duplicate rows
#     merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T', 'X', 'Y'])

#     # Calculate the value of 'logQ' column using np.log10 and np.inf
#     logQ = np.log10(merged_df['K']) + merged_df['n_invers'] * np.log10(merged_df['Y'])
#     merged_df['logQ_gas_g_per_100g_C'] = np.where((merged_df['Y'] >= merged_df['Ymin_ppmw']) & (merged_df['Y'] <= merged_df['Ymax_ppmw']),
#                                  logQ, 
#                                  -np.inf)
#     # Drop unnecessary columns and return the resulting dataframe
#     merged_df = merged_df.drop(['Unnamed: 0', 'K', 'n_invers', 'Ymin_ppmw', 'Ymax_ppmw'], axis=1)
#     return merged_df

# df_full = adsorption_capacity_gas(df_full, df_adsorption_capacity_gas)
# print(df_full)

                SMILES      MW        T    X    Y  logQ_gas_g_per_100g_C
0        BrC(Br)(Br)Br  331.63   148.00 -inf -inf                   -inf
3        BrC(Br)(Br)Br  331.63   150.00 -inf -inf                   -inf
6        BrC(Br)(Br)Br  331.63   186.15 -inf -inf                   -inf
9        BrC(Br)(Br)Br  331.63   199.95 -inf -inf                   -inf
12       BrC(Br)(Br)Br  331.63   200.00 -inf -inf                   -inf
...                ...     ...      ...  ...  ...                    ...
1450094        c1cocn1   69.06  1250.00 -inf -inf                   -inf
1450096        c1cocn1   69.06  1300.00 -inf -inf                   -inf
1450098        c1cocn1   69.06  1350.00 -inf -inf                   -inf
1450100        c1cocn1   69.06  1400.00 -inf -inf                   -inf
1450102        c1cocn1   69.06  1450.00 -inf -inf                   -inf

[687138 rows x 6 columns]


/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


2. adsorption capacity of liquid
we want to get Y as a function of K, n_invers, Xmin_ppmw, Xmax_ppmw

Q, Adsorption Capacity of Activated Carbon - Liquid
logQ_liq = logK + 1/n logX
Q g/100g C, Y ppmv

In [84]:
# def adsorption_capacity_liq(df_full, df_adsorption_capacity_liq):
#     # delete duplicate rows in df_adsorption_capacity_liq
#     df_adsorption_capacity_liq = df_adsorption_capacity_liq.drop_duplicates(subset='SMILES')
#     # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
#     merged_df = pd.merge(df_full, df_adsorption_capacity_liq, on='SMILES', how='left')
#     # drop duplicate rows
#     merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T', 'X', 'Y'])
#     # Calculate the value of 'logQ' column using np.log10 and np.inf
#     logQ = np.log10(merged_df['K']) + merged_df['n_invers'] * np.log10(merged_df['X'])
#     merged_df['logQ_liq_g_per_100g_C'] = np.where((merged_df['X'] >= merged_df['Xmin_ppmw']) & (merged_df['X'] <= merged_df['Xmax_ppmw']),
#                                  logQ, 
#                                  -np.inf)
    
#     # Drop unnecessary columns and return the resulting dataframe
#     merged_df = merged_df.drop(['Unnamed: 0', 'K', 'n_invers', 'Xmin_ppmw', 'Xmax_ppmw'], axis=1)
#     return merged_df

# df_full = adsorption_capacity_liq(df_full, df_adsorption_capacity_liq)
# print(df_full)


               SMILES      MW        T    X    Y  logQ_gas_g_per_100g_C  \
0       BrC(Br)(Br)Br  331.63   148.00 -inf -inf                   -inf   
1       BrC(Br)(Br)Br  331.63   150.00 -inf -inf                   -inf   
2       BrC(Br)(Br)Br  331.63   186.15 -inf -inf                   -inf   
3       BrC(Br)(Br)Br  331.63   199.95 -inf -inf                   -inf   
4       BrC(Br)(Br)Br  331.63   200.00 -inf -inf                   -inf   
...               ...     ...      ...  ...  ...                    ...   
687133        c1cocn1   69.06  1250.00 -inf -inf                   -inf   
687134        c1cocn1   69.06  1300.00 -inf -inf                   -inf   
687135        c1cocn1   69.06  1350.00 -inf -inf                   -inf   
687136        c1cocn1   69.06  1400.00 -inf -inf                   -inf   
687137        c1cocn1   69.06  1450.00 -inf -inf                   -inf   

        logQ_liq_g_per_100g_C  
0                        -inf  
1                        -inf  
2  

/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


3. vapour pressure from Antoine coefficients
we want to get log10P as a function of A, B, C, D, E and T

log10P, vapour pressure at temperature T

In [22]:
def vapour_pressure_from_antoine_coefficients(df_full, df_antoine_coefficients_mixed):
    # delete duplicate rows in df_antoine_coefficients_mixed
    #df_antoine_coefficients_mixed = df_antoine_coefficients_mixed.drop_duplicates(subset='SMILES')
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_antoine_coefficients_mixed, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # drop all rows where D can't be converted to float
    merged_df = merged_df.dropna(subset=['D'])
    # replace - with 0 in D and E columns
    merged_df['D'] = merged_df['D'].replace('-', '0')
    merged_df['E'] = merged_df['E'].replace('-', '0') 
    merged_df['D'] = merged_df['D'].astype(float)
    merged_df['E'] = merged_df['E'].astype(float)


    # Calculate the value of log10P
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # if SI is true, log10P = A + B/T + C * log10(T) + D * T + E * T^2
    # if SI is false, log10P = A - (B /(T+C))
    merged_df['log10P_mmHg'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        np.where(merged_df['SI'] == True,
                                 merged_df['A'] + merged_df['B']/merged_df['T'] + merged_df['C'] * np.log10(merged_df['T']) + merged_df['D'] * merged_df['T'] + merged_df['E'] * merged_df['T']**2,
                                 merged_df['A'] - (merged_df['B'] /(merged_df['T']+merged_df['C']))),
                                -np.inf)
    
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'D', 'E', 'Tmin', 'Tmax', 'SI'], axis=1)
    return merged_df

df_full = vapour_pressure_from_antoine_coefficients(df_full, df_antoine_coefficients_mixed)
# reset index
df_full = df_full.reset_index(drop=True)
print(df_full)

                SMILES   MW       T  log10P_mmHg
0        BrC(Br)(Br)Br -inf   148.0         -inf
1        BrC(Br)(Br)Br -inf   150.0         -inf
2        BrC(Br)(Br)Br -inf   160.0         -inf
3        BrC(Br)(Br)Br -inf   170.0         -inf
4        BrC(Br)(Br)Br -inf   180.0         -inf
...                ...  ...     ...          ...
1128591        c1cocn1 -inf  1450.0         -inf
1128592        c1cocn1 -inf  1460.0         -inf
1128593        c1cocn1 -inf  1470.0         -inf
1128594        c1cocn1 -inf  1480.0         -inf
1128595        c1cocn1 -inf  1490.0         -inf

[1128596 rows x 4 columns]


4. critical values
we want to get critical temperature Tc (K), critical pressure (bar), critical volume (cm3/mol), Zc and omega
??? Zc, omega???

In [23]:
def critical_values(df_full, df_critical_values):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_critical_values, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])

    # get new columns Tc, Pc, Vc, Zc, omega
    merged_df['Tc_K'] = merged_df['Tc'].astype(float)
    merged_df['Pc_bar'] = merged_df['Pc'].astype(float)
    merged_df['Vc_cm3_per_mol'] = merged_df['Vc'].astype(float)
    merged_df['Zc'] = merged_df['Zc'].astype(float)
    merged_df['omega'] = merged_df['omega'].astype(float)

    # set those columns to -np.inf when T != 298.15
    merged_df['Tc_K'] = np.where(merged_df['T'] == 298.15, merged_df['Tc_K'], -np.inf).astype(float)
    merged_df['Pc_bar'] = np.where(merged_df['T'] == 298.15, merged_df['Pc_bar'], -np.inf).astype(float)
    merged_df['Vc_cm3_per_mol'] = np.where(merged_df['T'] == 298.15, merged_df['Vc_cm3_per_mol'], -np.inf).astype(float)
    merged_df['Zc'] = np.where(merged_df['T'] == 298.15, merged_df['Zc'], -np.inf).astype(float)
    merged_df['omega'] = np.where(merged_df['T'] == 298.15, merged_df['omega'], -np.inf).astype(float)

    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'Tc', 'Pc', 'Vc'], axis=1)
    return merged_df

df_full = critical_values(df_full, df_critical_values)
print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1133827        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1133828        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1133829        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1133830        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1133831        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  
0                  -inf  
1          

4. density
we want to get density rho (g/cm3)

In [24]:
def get_density(df_full, df_density):
    # rename Tc column of df_density to Tc_density
    df_density = df_density.rename(columns={'Tc': 'Tc_density'})
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_density, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, density = A ( B^( -(1-T/Tc)^n ) ) )

    merged_df['rho_g_per_cm3'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] * (merged_df['B']**(-(1-merged_df['T']/merged_df['Tc_density'])**merged_df['n'])),
                        -np.inf)
    
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'n', 'Tc_density', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_density(df_full, df_density)
print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134065        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134066        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134067        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134068        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134069        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  
0                  -in

5. diffusion coefficient air
we want to get the diffusion coefficient in air in cm2/s as a function of A, B, C and T

In [25]:
def get_diffusion_coefficient_air(df_full, df_diffusion_coefficient_air):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_diffusion_coefficient_air, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, D_cm2_per_s = A + B*T + C*T^2

    merged_df['D_air_cm2_per_s'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_diffusion_coefficient_air(df_full, df_diffusion_coefficient_air)
print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1128591        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1128592        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1128593        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1128594        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1128595        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  
0    

6. diffusion coefficient water infinite dilution
we want to get the diffusion coefficient in water in cm2/s as a function of A, B, C and T

In [26]:
def get_diffusion_coefficient_water(df_full, df_diffusion_coefficient_water_infinite_dilution):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_diffusion_coefficient_water_infinite_dilution, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, D_cm2_per_s = 10^(A + B/T)

    merged_df['D_water_cm2_per_s'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        10**(merged_df['A'] + merged_df['B']/merged_df['T']),
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_diffusion_coefficient_water(df_full, df_diffusion_coefficient_water_infinite_dilution)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1128591        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1128592        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1128593        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1128594        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1128595        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

7. enthalpy of combustion
we want to get dHcomb (kJ/mol)

In [27]:
def get_combustion_enthalpy(df_full, df_enthalpy_of_combustion):
    # rename T column of df_enthalpy_of_combustion to T_standard
    df_enthalpy_of_combustion = df_enthalpy_of_combustion.rename(columns={'T': 'T_standard'})
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_enthalpy_of_combustion, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # get phase as phase_298K, if phase is gas, set to 0, if liquid, set to 1, if solid, set to 2
    #merged_df['phase_298K'] = merged_df['phase'].map({'gas': 0, 'liquid': 1, 'solid': 2})
    # if T is T_standard, return dHcomb_kJ_per_mol, else return -np.inf
    merged_df['dHcomb_kJ_per_mol'] = np.where(merged_df['T'] == 298.15,
                        merged_df['dHcomb_kJ_per_mol'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'phase', 'T_standard'], axis=1)
    return merged_df

df_full = get_combustion_enthalpy(df_full, df_enthalpy_of_combustion)

print(df_full)
# print contents of last column
print(df_full.iloc[:,-1])

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134779        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134780        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134781        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134782        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134783        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

8. enthalpy of formation
get enthalpy of formation as a function of A, B, C and T

In [28]:
def get_formation_enthalpy(df_full, df_enthalpy_of_formation):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_enthalpy_of_formation, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, dHform_kJ_per_mol = A + B*T + C*T^2

    merged_df['dHform_kJ_per_mol'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_formation_enthalpy(df_full, df_enthalpy_of_formation)
df_full[df_full['dHorm_kJ_per_mol'] != -np.inf].groupby('SMILES').first()

print(df_full)

KeyError: 'dHorm_kJ_per_mol'

9. enthalpy of fusion
get enthalpy of fusion (kJ/mol) as a function at the freezing temperature (K)

In [54]:
def get_fusion_enthalpy(df_full, df_enthalpy_of_fusion):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_enthalpy_of_fusion, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # rename T_freeze into T_freeze_K
    merged_df = merged_df.rename(columns={'T_freeze': 'T_freeze_K'})

    # set T_freeze_K to -np.inf when T!=298.15
    merged_df['T_freeze_K'] = np.where(merged_df['T'] == 298.15,
                        merged_df['T_freeze_K'],
                        -np.inf)
    # set dHfusion_kJ_per_mol to -np.inf when T!=298.15
    merged_df['dHfusion_kJ_per_mol'] = np.where(merged_df['T'] == 298.15,
                        merged_df['dHfusion_kJ_per_mol'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0'], axis=1)
    return merged_df

df_full = get_fusion_enthalpy(df_full, df_enthalpy_of_fusion)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134065        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134066        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134067        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134068        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134069        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

10. enthalpy of vaporization
get the enthalpy of vaporization (kJ/mol) as a function of A, Tc, n

In [55]:
def get_vaporization_enthalpy(df_full, df_enthalpy_of_vaporization):
    # rename Tc in df_enthalpy_of_vaporization to Tc_vap
    df_enthalpy_of_vaporization = df_enthalpy_of_vaporization.rename(columns={'Tc': 'Tc_vap'})
    # rename T_boil to T_boil_K
    df_enthalpy_of_vaporization = df_enthalpy_of_vaporization.rename(columns={'T_boil': 'T_boil_K'})
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_enthalpy_of_vaporization, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, dHvap_kJ_per_mol = A * ( 1 - T/Tc )^n
    merged_df['dHvap_kJ_per_mol'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] * (1 - merged_df['T']/merged_df['Tc_vap'])**merged_df['n'],
                        -np.inf)
    
    # set T_boil_K to -np.inf when T!=298.15
    merged_df['T_boil_K'] = np.where(merged_df['T'] == 298.15,
                        merged_df['T_boil_K'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'Tc_vap', 'n', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_vaporization_enthalpy(df_full, df_enthalpy_of_vaporization)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

11. entropy of formation
get entropy of formation (J/mol*K) of ideal gas as a function of A, B, C and T

In [56]:
def get_formation_entropy(df_full, df_entropy_of_formation):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_entropy_of_formation, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, S_form_J_per_mol_per_K = A + B/T + C*T
    merged_df['S_form_J_per_mol_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']/merged_df['T'] + merged_df['C']*merged_df['T'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_formation_entropy(df_full, df_entropy_of_formation)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

12. entropy of gas
get entropy of ideal gas (J/mol*K) as a funciton of A, B, C, T

In [57]:
def get_gas_entropy(df_full, df_entropy_of_gas):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_entropy_of_gas, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, S_gas_J_per_mol_per_K = A + B*T + C*T^2
    merged_df['S_gas_J_per_mol_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

dffull = get_gas_entropy(df_full, df_entropy_of_gas)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  D_air_cm2_per_s  D_wate

13. experimental hazards
get common experimental hazards LOE_upper (%), LOE_lower (%), flashpoint_K (K), auto ignition temperature AIT_K (K)

In [58]:
def get_exp_hazards(df_full, df_experimental_hazards):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_experimental_hazards, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # set experimental hazards to -np.inf when T!=298.15
    merged_df['LOE_upper'] = np.where(merged_df['T'] == 298.15,
                        merged_df['LOE_upper'],
                        -np.inf)
    merged_df['LOE_lower'] = np.where(merged_df['T'] == 298.15,
                        merged_df['LOE_lower'],
                        -np.inf)
    merged_df['flashpoint_K'] = np.where(merged_df['T'] == 298.15,
                        merged_df['flashpoint_K'],
                        -np.inf)
    merged_df['AIT_K'] = np.where(merged_df['T'] == 298.15,
                        merged_df['AIT_K'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0'], axis=1)
    return merged_df

df_full = get_exp_hazards(df_full, df_experimental_hazards)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1132399        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1132400        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1132401        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1132402        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1132403        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dHform_kJ_per_mol 

14. gibbs free energy
get Gibbs free energy of ideal gas formation (kJ/mol) as a funciton of A, B, C and T

In [59]:
def get_formation_gibbs_free_energy(df_full, df_gibbs_free_energy_of_formation):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_gibbs_free_energy_of_formation, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, dGform_kJ_per_mol = A + B*T + C*T^2
    merged_df['dGform_kJ_per_mol'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_formation_gibbs_free_energy(df_full, df_gibbs_free_energy_of_formation)
print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  T_freeze_K  dHfusi

15. heat capacity gas
get heat capacity of gas (J/mol*K) as a function of A, B, C, D, E, T

In [60]:
def get_heat_capacity_gas(df_full, df_heat_capacity_gas):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_heat_capacity_gas, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, Cp_gas_J_per_mol_per_K = A + B*T + C*T^2 + D*T^3 + E*T^4
    merged_df['Cp_gas_J_per_mol_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2 + merged_df['D']*merged_df['T']**3 + merged_df['E']*merged_df['T']**4,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'D', 'E', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_heat_capacity_gas(df_full, df_heat_capacity_gas)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dHfusion_kJ_per_mo

16. heat capacity liquid
get heat capacity of liquid (J/mol*K) as a function of A, B, C, D, T

In [61]:
def get_heat_capacity_liq(df_full, df_heat_capacity_liq):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_heat_capacity_liq, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, Cp_liq_J_per_mol_per_K = A + B*T + C*T^2 + D*T^3
    merged_df['Cp_liq_J_per_mol_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2 + merged_df['D']*merged_df['T']**3,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'D', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_heat_capacity_liq(df_full, df_heat_capacity_liq)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134779        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134780        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134781        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134782        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134783        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  T_boil_K  dHvap_kJ

17. heat capacity solid
get heat capacity of solid (J/mol*K) as a function of A, B, C, T

In [62]:
def get_heat_capacity_sol(df_full, df_heat_capacity_sol):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_heat_capacity_sol, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # convert column C to float
    #replace - with 0
    merged_df['C'] = merged_df['C'].replace('-', 0)
    merged_df['C'] = merged_df['C'].astype(float)
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, Cp_sol_J_per_mol_per_K = A + B*T + C*T^2
    merged_df['Cp_sol_J_per_mol_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_heat_capacity_sol(df_full, df_heat_capacity_sol)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134779        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134780        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134781        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134782        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134783        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dHvap_kJ_per_mol  

18. helmholtz free energy
get helmholtz free energy of ideal gas formation (kJ/mol) as a funciton of A, B, C and T

In [63]:
def get_formation_helmholtz_free_energy(df_full, df_helmholtz_free_energy_of_formation):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_helmholtz_free_energy_of_formation, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, dAform_kJ_per_mol = A + B*T + C*T^2
    merged_df['dAform_kJ_per_mol'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_formation_helmholtz_free_energy(df_full, df_helmholtz_free_energy_of_formation)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  S_form_J_per_mol_p

19. henry constant
get henry constant at specific temperature from experimental data

In [64]:
def get_henry_constant(df_full, df_henry_constant):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_henry_constant, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # rename column henry_atm_per_m3_per_mol to henry_atm_m3_per_mol
    merged_df = merged_df.rename(columns={'henry_atm_per_m3_per_mol': 'henry_atm_m3_per_mol'})
    
    # if T matches T_degK, return henry_atm_per_molfrac, else return -np.inf
    merged_df['henry_atm_m3_per_mol'] = np.where(merged_df['T'] == merged_df['T_degK'],
                        merged_df['henry_atm_m3_per_mol'],
                        -np.inf)
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'T_freeze', 'T_boil', 'henry_atm_per_molfrac', 'T_degK'], axis=1)
    return merged_df

df_full = get_henry_constant(df_full, df_henry_constant)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1130495        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1130496        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1130497        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1130498        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1130499        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  LOE_lower  LOE_upp

20. internal energy of formation
get internal free energy of ideal gas formation (kJ/mol) as a funciton of A, B, C and T

In [65]:
def get_formation_internal_free_energy(df_full, df_internal_energy_of_formation):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_internal_energy_of_formation, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    # calculate density
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, dUform_kJ_per_mol = A + B*T + C*T^2
    merged_df['dUform_kJ_per_mol'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_formation_internal_free_energy(df_full, df_internal_energy_of_formation)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  LOE_upper  flashpo

21. refractive index
get refractive index in nD at a specific temperature

In [66]:
def get_refractive_index(df_full, df_refractive_index):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_refractive_index, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # if T matches T_degK, return RI_at_T_nD for this row, else return -np.inf
    merged_df['RI_nD'] = np.where(merged_df['T'] == merged_df['T_degK'],
                        merged_df['RI_at_T_nD'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'T_freeze', 'T_boil', 'T_degK', 'RI_at_T_nD'], axis=1)
    return merged_df

df_full = get_refractive_index(df_full, df_refractive_index)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1132637        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1132638        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1132639        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1132640        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1132641        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  flashpoint_K  AIT_

22. solubility in octanol and water
get solubility in water and octanol and the octanol-water partition coefficient (rater only the partition coefficient)

In [67]:
#;SMILES;T_freeze;T_boil;henry_atm_per_molfrac;henry_atm_per_m3_per_mol;T_degK
#### T freeze? T boil?
def get_octanol_water_partition_coeff(df_full, df_solubility_octanol_water):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_solubility_octanol_water, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # if T matches T_degK, return logP_octanol_water, else return -np.inf
    merged_df['logP_octanol_water'] = np.where(merged_df['T'] == merged_df['T_degK'],
                        merged_df['logP_octanol_water'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'solubility_octanol_at_T_ppmw', 'solubility_in_octanol_at_T_ppmm', 'T_degK'], axis=1)
    return merged_df

df_full = get_octanol_water_partition_coeff(df_full, df_solubility_octanol_water)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1130733        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1130734        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1130735        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1130736        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1130737        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  AIT_K  dGform_kJ_p

23. solubility parameters
get the solubility parameterdipole moment (D), VdW area (cm2/mol) VdW volume (cm3/mol), radius of gyration (Angstrom)
skip  (sqrt_J_per_cm3), liquid volume (cm3_per_mol) for now

In [68]:
def get_dipole_vdw(df_full, df_solubility_parameters):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_solubility_parameters, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # get dipole_moment_D from dipole_moment, VdW_area_cm2_per_mol from VdW_area, VdW_volume_cm3_per_mol from VdW_volume, r_gyration_A from r_gyration
    merged_df['dipole_moment_D'] = merged_df['dipole_moment']
    merged_df['VdW_area_cm2_per_mol'] = merged_df['VdW_area']
    merged_df['VdW_volume_cm3_per_mol'] = merged_df['VdW_volume']
    merged_df['r_gyration_A'] = merged_df['r_gyration']

    # set values to -np.inf when T != 298.15 K
    merged_df['dipole_moment_D'] = np.where(merged_df['T'] != 298.15,
                        -np.inf,
                        merged_df['dipole_moment_D'])
    merged_df['VdW_area_cm2_per_mol'] = np.where(merged_df['T'] != 298.15,
                        -np.inf,
                        merged_df['VdW_area_cm2_per_mol'])
    merged_df['VdW_volume_cm3_per_mol'] = np.where(merged_df['T'] != 298.15,
                        -np.inf,
                        merged_df['VdW_volume_cm3_per_mol'])
    merged_df['r_gyration_A'] = np.where(merged_df['T'] != 298.15,
                        -np.inf,
                        merged_df['r_gyration_A'])
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'solubility_parameter', 'V_liq', 'T_degK', 'dipole_moment', 'VdW_area', 'VdW_volume', 'r_gyration'], axis=1)
    return merged_df

df_full = get_dipole_vdw(df_full, df_solubility_parameters)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134779        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134780        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134781        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134782        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134783        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  Cp_sol_J_per_mol_p

24. surface tension
get surface tension in dyne/cm as a function of A, Tc, n

In [69]:
def get_surface_tension(df_full, df_surface_tension):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_surface_tension, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, sigma_dyne_per_cm = A * (1-T/Tc)^n
    merged_df['sigma_dyne_per_cm'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] * (1-merged_df['T']/merged_df['Tc'])**merged_df['n'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'Tc', 'n', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_surface_tension(df_full, df_surface_tension)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134779        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134780        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134781        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134782        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134783        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dAform_kJ_per_mol 

25. thermal conductivity gas
get thermal conductivity gas (W/mK) as a function of A, B, C, T

In [70]:
def get_thermal_conductivity_gas(df_full, df_thermal_conductivity_gas):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_thermal_conductivity_gas, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, thermal_cond_W_per_mK = A + B*T + C*T^2
    merged_df['thermal_cond_gas_W_per_mK'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_thermal_conductivity_gas(df_full, df_thermal_conductivity_gas)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134065        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134066        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134067        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134068        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134069        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  henry_atm_m3_per_m

26. thermal conductivity liquid
get thermal conductivity liquid (W/mK) as a function of A, B, C, T

In [71]:
def get_thermal_conductivity_liq(df_full, df_thermal_conductivity_liq):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_thermal_conductivity_liq, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, thermal_cond_W_per_mK = A + B*T + C*T^2
    merged_df['thermal_cond_liq_W_per_mK'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_thermal_conductivity_liq(df_full, df_thermal_conductivity_liq)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134065        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134066        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134067        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134068        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134069        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dUform_kJ_per_mol 

27. thermal expansion coefficient of liquid
get thermal expansion coefficient (1/K) as a function of a, Tc, m

In [72]:
def get_thermal_expansion_coef(df_full, df_thermal_expansion_liq):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_thermal_expansion_liq, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, thermal_exp_coef_per_K = a * (1 - T/Tc)^m
    merged_df['thermal_exp_coef_liq_per_K'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['a'] * (1-merged_df['T']/merged_df['Tc'])**merged_df['m'],
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'a', 'Tc', 'm', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_thermal_expansion_coef(df_full, df_thermal_expansion_liq)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  RI_nD  logP_octano

28. viscosity of gas
get µ gas (cP) as a function of A, B, C, D, T

In [73]:
def get_gas_viscosity(df_full, df_viscosity_gas):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_viscosity_gas, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, viscosity_gas_cP = A + B*T + C*T^2
    merged_df['viscosity_gas_cP'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        merged_df['A'] + merged_df['B']*merged_df['T'] + merged_df['C']*merged_df['T']**2,
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_gas_viscosity(df_full, df_viscosity_gas)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134303        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134304        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134305        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134306        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134307        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  logP_octanol_water

29. viscosity of liquid
get µ gas (cP) as a function of A, B, C, D, T

In [74]:
def get_liq_viscosity(df_full, df_viscosity_liq):
    # Merge df_full with df_adsorption_capacity_gas on 'SMILES' column
    merged_df = pd.merge(df_full, df_viscosity_liq, on='SMILES', how='left')
    # drop duplicate rows
    merged_df = merged_df.drop_duplicates(subset=['SMILES', 'T'])
    
    # check if T is between Tmin and Tmax, if not, return -np.inf
    # else, viscosity_liq_cP = 10^(A + B/T + C*T + D*T^2)
    merged_df['viscosity_liq_cP'] = np.where((merged_df['T'] >= merged_df['Tmin']) & (merged_df['T'] <= merged_df['Tmax']),
                        10**(merged_df['A'] + merged_df['B']/merged_df['T'] + merged_df['C']*merged_df['T'] + merged_df['D']*merged_df['T']**2),
                        -np.inf)
    
    # Drop unnecessary columns and return the resulting dataframe
    merged_df = merged_df.drop(['Unnamed: 0', 'A', 'B', 'C', 'D', 'Tmin', 'Tmax'], axis=1)
    return merged_df

df_full = get_liq_viscosity(df_full, df_viscosity_liq)

print(df_full)

                SMILES   MW       T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0        BrC(Br)(Br)Br -inf   148.0         -inf -inf   -inf  -inf    -inf   
1        BrC(Br)(Br)Br -inf   150.0         -inf -inf   -inf  -inf    -inf   
2        BrC(Br)(Br)Br -inf   160.0         -inf -inf   -inf  -inf    -inf   
3        BrC(Br)(Br)Br -inf   170.0         -inf -inf   -inf  -inf    -inf   
4        BrC(Br)(Br)Br -inf   180.0         -inf -inf   -inf  -inf    -inf   
...                ...  ...     ...          ...  ...    ...   ...     ...   
1134541        c1cocn1 -inf  1450.0         -inf -inf   -inf  -inf    -inf   
1134542        c1cocn1 -inf  1460.0         -inf -inf   -inf  -inf    -inf   
1134543        c1cocn1 -inf  1470.0         -inf -inf   -inf  -inf    -inf   
1134544        c1cocn1 -inf  1480.0         -inf -inf   -inf  -inf    -inf   
1134545        c1cocn1 -inf  1490.0         -inf -inf   -inf  -inf    -inf   

         Vc_cm3_per_mol  rho_g_per_cm3  ...  dipole_moment_D  \

In [75]:
print(df_full.columns)
print(df_full.head())

# print how many NaN values are in each column
print(df_full.isnull().sum())
# replace all NaN values with -np.inf
df_full = df_full.replace(np.nan, -np.inf)
# replace all empty strings with -np.inf
df_full = df_full.replace('  ', -np.inf)
df_full = df_full.replace('', -np.inf)
df_full = df_full.replace(' ', -np.inf)
print(df_full.isnull().sum())
# print how many -np.inf values are in each column
print(df_full.isin([-np.inf]).sum())

print(len(df_full))

# save as full_dataset_for_training.csv with sep = ;, save the column names
df_full.to_csv('full_dataset_for_training.csv', sep=';', index=False)

Index(['SMILES', 'MW', 'T', 'log10P_mmHg', 'Zc', 'omega', 'Tc_K', 'Pc_bar',
       'Vc_cm3_per_mol', 'rho_g_per_cm3', 'D_air_cm2_per_s',
       'D_water_cm2_per_s', 'dHcomb_kJ_per_mol', 'dHform_kJ_per_mol',
       'T_freeze_K', 'dHfusion_kJ_per_mol', 'T_boil_K', 'dHvap_kJ_per_mol',
       'S_form_J_per_mol_per_K', 'LOE_lower', 'LOE_upper', 'flashpoint_K',
       'AIT_K', 'dGform_kJ_per_mol', 'Cp_gas_J_per_mol_per_K',
       'Cp_liq_J_per_mol_per_K', 'Cp_sol_J_per_mol_per_K', 'dAform_kJ_per_mol',
       'henry_atm_m3_per_mol', 'dUform_kJ_per_mol', 'RI_nD',
       'logP_octanol_water', 'dipole_moment_D', 'VdW_area_cm2_per_mol',
       'VdW_volume_cm3_per_mol', 'r_gyration_A', 'sigma_dyne_per_cm',
       'thermal_cond_gas_W_per_mK', 'thermal_cond_liq_W_per_mK',
       'thermal_exp_coef_liq_per_K', 'viscosity_gas_cP', 'viscosity_liq_cP'],
      dtype='object')
          SMILES   MW      T  log10P_mmHg   Zc  omega  Tc_K  Pc_bar  \
0  BrC(Br)(Br)Br -inf  148.0         -inf -inf   -inf  -inf 

In [40]:
df_full = pd.read_csv('full_dataset_for_training.csv', sep=';')

#print(df_full.head())

# add a new column 'c0' which is 0
df_full['c0'] = 0

# rename T to c1
df_full = df_full.rename(columns={'SMILES': 'SMILES0', 'T': 'c1'})

columns_list_to_take_logarithm_from = ['MW', 'Pc_bar',
       'Vc_cm3_per_mol', 'D_air_cm2_per_s',
       'D_water_cm2_per_s', 'dHcomb_kJ_per_mol', 'dHform_kJ_per_mol', 'LOE_lower', 'LOE_upper',
       'AIT_K', 'Cp_gas_J_per_mol_per_K',
       'dipole_moment_D', 'VdW_area_cm2_per_mol',
       'VdW_volume_cm3_per_mol', 'sigma_dyne_per_cm',
       'thermal_cond_gas_W_per_mK', 'thermal_cond_liq_W_per_mK',
       'thermal_exp_coef_liq_per_K', 'viscosity_gas_cP',
       'viscosity_liq_cP']

# loop through each column
for column in df_full.columns:
    # Check if all values are non-negative (ignoring NaNs)
    # And the values span at least two orders of magnitude
    min_val = df_full[column].min()
    max_val = df_full[column].max()
    
    if min_val >= 0 and max_val / min_val >= 100: 
        try:
            df_full['log_'+column] = df_full[column].apply(lambda x: math.log(x) if x > 0 else -np.inf) 
            del df_full[column]
            print(f"successful for {column}")
        except:
            print(f"failed for {column}")
            continue
    else:
        print(f"conditions not met for {column}")

# reorder columns so that SMILES0, c0, c1 are the first three columns
df_full = df_full[['SMILES0', 'c0', 'c1'] + [col for col in df_full.columns if col not in ['SMILES0', 'c0', 'c1']]]

# save as full_dataset_for_training.csv with sep = ;, save the column names
df_full.to_csv('full_dataset_for_training_logs.csv', sep=';', index=False)

print(df_full.columns)


successful for MW
successful for Pc_bar
successful for Vc_cm3_per_mol
successful for D_air_cm2_per_s
successful for D_water_cm2_per_s
failed for dHcomb_kJ_per_mol
failed for dHform_kJ_per_mol
successful for LOE_lower
successful for LOE_upper
successful for AIT_K
successful for Cp_gas_J_per_mol_per_K
failed for dipole_moment_D
successful for VdW_area_cm2_per_mol
successful for VdW_volume_cm3_per_mol
failed for sigma_dyne_per_cm
successful for thermal_cond_gas_W_per_mK
successful for thermal_cond_liq_W_per_mK
successful for thermal_exp_coef_liq_per_K
successful for viscosity_gas_cP
successful for viscosity_liq_cP
Index(['SMILES0', 'c0', 'c1', 'log10P_mmHg', 'Zc', 'omega', 'Tc_K',
       'rho_g_per_cm3', 'dHcomb_kJ_per_mol', 'dHform_kJ_per_mol', 'T_freeze_K',
       'dHfusion_kJ_per_mol', 'T_boil_K', 'dHvap_kJ_per_mol',
       'S_form_J_per_mol_per_K', 'flashpoint_K', 'dGform_kJ_per_mol',
       'Cp_liq_J_per_mol_per_K', 'Cp_sol_J_per_mol_per_K', 'dAform_kJ_per_mol',
       'henry_atm_m3_

In [79]:
# load the full dataset
df_full = pd.read_csv('full_dataset_for_training_logs.csv', sep=';')

# create a dictionary that maps
# SMILES to SMILES0
# c0 set to 0 (composition)
# T to c1
# MW to y0
# and all other columns starting from y1
dict_columns = {}
for i, col in enumerate(df_full.columns):
    if i < 3:
        continue
    else:
        dict_columns[col] = 'y' + str(i-3)

# switch keys and values
dict_columns = {v: k for k, v in dict_columns.items()}
print(dict_columns)
# convert dict to dataframe
df_dict_columns = pd.DataFrame.from_dict(dict_columns, orient='index')
# save as dict_columns.csv with sep = ;, save the column names
df_dict_columns.to_csv('dict_columns.csv', index=True, header=False)

{'y0': 'log10P_mmHg', 'y1': 'Zc', 'y2': 'omega', 'y3': 'Tc_K', 'y4': 'rho_g_per_cm3', 'y5': 'T_freeze_K', 'y6': 'dHfusion_kJ_per_mol', 'y7': 'T_boil_K', 'y8': 'dHvap_kJ_per_mol', 'y9': 'S_form_J_per_mol_per_K', 'y10': 'flashpoint_K', 'y11': 'dGform_kJ_per_mol', 'y12': 'Cp_liq_J_per_mol_per_K', 'y13': 'Cp_sol_J_per_mol_per_K', 'y14': 'dAform_kJ_per_mol', 'y15': 'henry_atm_m3_per_mol', 'y16': 'dUform_kJ_per_mol', 'y17': 'RI_nD', 'y18': 'logP_octanol_water', 'y19': 'r_gyration_A', 'y20': 'log_MW', 'y21': 'log_Pc_bar', 'y22': 'log_Vc_cm3_per_mol', 'y23': 'log_D_air_cm2_per_s', 'y24': 'log_D_water_cm2_per_s', 'y25': 'log_dHcomb_kJ_per_mol', 'y26': 'log_dHform_kJ_per_mol', 'y27': 'log_LOE_lower', 'y28': 'log_LOE_upper', 'y29': 'log_AIT_K', 'y30': 'log_Cp_gas_J_per_mol_per_K', 'y31': 'log_dipole_moment_D', 'y32': 'log_VdW_area_cm2_per_mol', 'y33': 'log_VdW_volume_cm3_per_mol', 'y34': 'log_sigma_dyne_per_cm', 'y35': 'log_thermal_cond_gas_W_per_mK', 'y36': 'log_thermal_cond_liq_W_per_mK', 'y37'

In [83]:
df_full = pd.read_csv('full_dataset_for_training_logs.csv', sep=";")
# switch keys and values
dict_columns = {v: k for k, v in dict_columns.items()}
df_full = df_full.rename(columns=dict_columns)
print(df_full)

               SMILES0  c0      c1   y0   y1   y2   y3   y4   y5   y6  ...  \
0        BrC(Br)(Br)Br   0   148.0 -inf -inf -inf -inf -inf -inf -inf  ...   
1        BrC(Br)(Br)Br   0   150.0 -inf -inf -inf -inf -inf -inf -inf  ...   
2        BrC(Br)(Br)Br   0   160.0 -inf -inf -inf -inf -inf -inf -inf  ...   
3        BrC(Br)(Br)Br   0   170.0 -inf -inf -inf -inf -inf -inf -inf  ...   
4        BrC(Br)(Br)Br   0   180.0 -inf -inf -inf -inf -inf -inf -inf  ...   
...                ...  ..     ...  ...  ...  ...  ...  ...  ...  ...  ...   
1128591        c1cocn1   0  1450.0 -inf -inf -inf -inf -inf -inf -inf  ...   
1128592        c1cocn1   0  1460.0 -inf -inf -inf -inf -inf -inf -inf  ...   
1128593        c1cocn1   0  1470.0 -inf -inf -inf -inf -inf -inf -inf  ...   
1128594        c1cocn1   0  1480.0 -inf -inf -inf -inf -inf -inf -inf  ...   
1128595        c1cocn1   0  1490.0 -inf -inf -inf -inf -inf -inf -inf  ...   

              y30  y31  y32  y33  y34  y35  y36  y37  y38  y39 

In [84]:
# Create a list of column names to exclude
exclude_cols = ['SMILES0', 'c0', 'c1']

# Define a lambda function to modify each column
modify_col = lambda col: (col - col[col != -np.inf].mean()) / col[col != -np.inf].std() if col.name not in exclude_cols else col

# Apply the lambda function to all columns in df_full
df_full = df_full.apply(modify_col, axis=0)

print(df_full.head(20))

df_full.to_csv("full_dataset_for_training_formatted_all_y.csv")


          SMILES0  c0      c1   y0   y1   y2   y3   y4   y5   y6  ...  y30  \
0   BrC(Br)(Br)Br   0  148.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
1   BrC(Br)(Br)Br   0  150.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
2   BrC(Br)(Br)Br   0  160.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
3   BrC(Br)(Br)Br   0  170.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
4   BrC(Br)(Br)Br   0  180.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
5   BrC(Br)(Br)Br   0  186.15 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
6   BrC(Br)(Br)Br   0  190.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
7   BrC(Br)(Br)Br   0  199.95 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
8   BrC(Br)(Br)Br   0  200.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
9   BrC(Br)(Br)Br   0  210.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
10  BrC(Br)(Br)Br   0  220.00 -inf -inf -inf -inf -inf -inf -inf  ... -inf   
11  BrC(Br)(Br)Br   0  230.00 -inf -inf -inf -inf -inf -inf -inf

In [85]:
print(df_full.columns)
print(df_full.head())
print(df_full.describe())

Index(['SMILES0', 'c0', 'c1', 'y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7',
       'y8', 'y9', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17',
       'y18', 'y19', 'y20', 'y21', 'y22', 'y23', 'y24', 'y25', 'y26', 'y27',
       'y28', 'y29', 'y30', 'y31', 'y32', 'y33', 'y34', 'y35', 'y36', 'y37',
       'y38', 'y39'],
      dtype='object')
         SMILES0  c0     c1   y0   y1   y2   y3   y4   y5   y6  ...  y30  y31  \
0  BrC(Br)(Br)Br   0  148.0 -inf -inf -inf -inf -inf -inf -inf  ... -inf -inf   
1  BrC(Br)(Br)Br   0  150.0 -inf -inf -inf -inf -inf -inf -inf  ... -inf -inf   
2  BrC(Br)(Br)Br   0  160.0 -inf -inf -inf -inf -inf -inf -inf  ... -inf -inf   
3  BrC(Br)(Br)Br   0  170.0 -inf -inf -inf -inf -inf -inf -inf  ... -inf -inf   
4  BrC(Br)(Br)Br   0  180.0 -inf -inf -inf -inf -inf -inf -inf  ... -inf -inf   

   y32  y33  y34  y35  y36  y37  y38  y39  
0 -inf -inf -inf -inf -inf -inf -inf -inf  
1 -inf -inf -inf -inf -inf -inf -inf -inf  
2 -inf -inf -inf -inf -inf -in

/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning

              c0            c1            y0            y1            y2  \
count  1128596.0  1.128596e+06  1.128596e+06  1.128596e+06  1.128596e+06   
mean         0.0  5.991855e+02          -inf          -inf          -inf   
std          0.0  3.884626e+02           NaN           NaN           NaN   
min          0.0  1.480000e+02          -inf          -inf          -inf   
25%          0.0  2.978000e+02           NaN           NaN           NaN   
50%          0.0  3.820000e+02           NaN           NaN           NaN   
75%          0.0  9.000000e+02 -1.218530e+00           NaN           NaN   
max          0.0  1.490000e+03  2.608359e+00  8.216015e+00  7.524723e+00   

                 y3            y4            y5            y6            y7  \
count  1.128596e+06  1.128596e+06  1.128596e+06  1.128596e+06  1.128596e+06   
mean           -inf          -inf          -inf          -inf          -inf   
std             NaN           NaN           NaN           NaN           NaN   

/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/local/home/mlederbauer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning